In [1]:
import numpy as np
import datetime
import joblib

In [9]:
import requests
import pandas as pd
from requests.structures import CaseInsensitiveDict

url = "https://smackbang-image-w76hg6ifha-ew.a.run.app/matches"

headers = CaseInsensitiveDict()
headers["accept"] = "application/json"

origin_one = 'NRT'
origin_two = 'SYD'
departure_date = '01/04/2022'
continent='AS'

query_string = {'origin_one':origin_one, 'origin_two':origin_two, 'departure_date':departure_date, 
                'continent':continent, 'return_date':'', 'currency':'USD'}

result = requests.get(url, headers=headers, params=query_string).json()
                
matches_df = pd.DataFrame(result)

In [10]:
matches_df.head()

,price_Sydney,price_Tokyo,duration_Sydney,duration_Tokyo,total_stops_Sydney,total_stops_Tokyo,distance_Sydney,distance_Tokyo,local_departure_Sydney,local_departure_Tokyo,local_arrival_Sydney,local_arrival_Tokyo,deep_link_Sydney,deep_link_Tokyo,combined_price_,combined_duration_,lat_,lon_
Hanoi,381,377,18.00,6.33,1,0,7798.15,3723.95,2022-04-01T13:25:00.000Z,2022-04-01T09:15:00.000Z,2022-04-02T11:25:00.000Z,2022-04-01T13:35:00.000Z,https://www.kiwi.com/deep?from=SYD&to=HAN&flig...,https://www.kiwi.com/deep?from=NRT&to=HAN&flig...,758,24.33,21.221201,105.806999
Bangkok,355,419,9.83,6.50,0,0,7523.15,4653.65,2022-04-01T10:50:00.000Z,2022-04-01T12:00:00.000Z,2022-04-01T16:40:00.000Z,2022-04-01T16:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=BKK&flig...,https://www.kiwi.com/deep?from=NRT&to=BKK&flig...,774,16.33,13.681100,100.747002
Taipei,380,416,19.00,3.67,1,0,7293.46,2183.46,2022-04-01T13:25:00.000Z,2022-04-01T13:50:00.000Z,2022-04-02T05:45:00.000Z,2022-04-01T16:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=TPE&flig...,https://www.kiwi.com/deep?from=NRT&to=TPE&flig...,796,22.67,25.077700,121.233002
Manila,479,341,8.33,4.75,0,0,6269.74,3054.20,2022-04-01T11:15:00.000Z,2022-04-01T13:10:00.000Z,2022-04-01T16:50:00.000Z,2022-04-01T17:30:00.000Z,https://www.kiwi.com/deep?from=SYD&to=MNL&flig...,https://www.kiwi.com/deep?from=NRT&to=MNL&flig...,820,13.08,14.508600,121.019997
Fukuoka,739,104,13.08,2.08,1,0,7823.25,940.88,2022-04-01T21:45:00.000Z,2022-04-01T07:10:00.000Z,2022-04-02T08:50:00.000Z,2022-04-01T09:15:00.000Z,https://www.kiwi.com/deep?from=SYD&to=FUK&flig...,https://www.kiwi.com/deep?from=NRT&to=FUK&flig...,843,15.16,33.585899,130.451004


In [338]:
city_one = 'Sydney'
city_two = 'Tokyo'

In [339]:
df1 = matches_df.iloc[:,2:12:2]

In [340]:
df1

,duration_Sydney,total_stops_Sydney,distance_Sydney,local_departure_Sydney,local_arrival_Sydney
Hanoi,18.00,1,7798.15,2022-04-01T13:25:00.000Z,2022-04-02T11:25:00.000Z
Bangkok,9.83,0,7523.15,2022-04-01T10:50:00.000Z,2022-04-01T16:40:00.000Z
Taipei,19.00,1,7293.46,2022-04-01T13:25:00.000Z,2022-04-02T05:45:00.000Z
Manila,8.33,0,6269.74,2022-04-01T11:15:00.000Z,2022-04-01T16:50:00.000Z
Fukuoka,13.08,1,7823.25,2022-04-01T21:45:00.000Z,2022-04-02T08:50:00.000Z
Singapore,8.25,0,6301.43,2022-04-01T08:15:00.000Z,2022-04-01T16:50:00.000Z
Osaka,12.33,1,7790.50,2022-04-01T21:45:00.000Z,2022-04-02T08:05:00.000Z
Kuala Lumpur,8.83,0,6591.75,2022-04-01T06:25:00.000Z,2022-04-01T20:15:00.000Z
Ho Chi Minh City,8.92,0,6853.68,2022-04-01T07:00:00.000Z,2022-04-01T16:10:00.000Z
Da Nang,21.17,1,7190.96,2022-04-01T16:00:00.000Z,2022-04-02T15:20:00.000Z


In [341]:
# convert local dearture to datetime
df1[f'local_departure_{city_one}'] = pd.to_datetime(df1[f'local_departure_{city_one}'])

In [342]:
# convert local arrival to datetime
df1[f'local_arrival_{city_one}'] = pd.to_datetime(df1[f'local_arrival_{city_one}'])

In [343]:
# create and format Date_of_Journey column based on local_departure
df1['Date_of_Journey'] = df1[f'local_departure_{city_one}'].dt.date[0].strftime("%d/%m/%Y")

In [344]:
# create and format Dep_Time column based on local_departure
df1["Dep_Time"] = df1[f"local_departure_{city_one}"].apply(lambda x: x.strftime("%H:%M"))

In [345]:
# create and format Arrival_time column based on local_arrival
df1["Arrival_Time"] = df1[f"local_arrival_{city_one}"].apply(lambda x: x.strftime("%H:%M"))

In [346]:
# create and format Duration 
df1['Duration'] = df1[f"duration_{city_one}"].apply(lambda x: f"{int(x)}h {(round(x*60)%60)}m")

In [348]:
# Convert Total_Stops_city_one to strings
df1[f'total_stops_{city_one}'] = df1[f'total_stops_{city_one}'].apply(str)

In [349]:
# create Source Column based on city_one
df1['Source'] = city_one

In [350]:
# create placeholder column for Additional_Info. This column will be dropped during preproc pipeline
df1['Additional_Info'] = "No info"

In [351]:
# create placeholder column for Airline. This column will be dropped during preproc pipeline
df1["Airline"] = "Multiple Carriers"

In [352]:
airports_df = pd.read_csv('../data/airport_codes.csv')

In [353]:
#creating empty lat,long columns type float64
df1['origin_one_latitude'] = np.nan
df1['origin_one_longitude'] = np.nan

# grabbing lat,lon from airports df
for city in list(df1.index):
    if city in list(airports_df['city']):
        df1.loc[city,'origin_one_latitude']=list(airports_df[airports_df['city']==city]['lat'])[0]
        df1.loc[city,'origin_one_longitude']=list(airports_df[airports_df['city']==city]['lon'])[0]

In [354]:
#creating empty lat,long columns type float64
df1['origin_one_latitude'] = np.nan
df1['origin_one_longitude'] = np.nan

# grabbing lat,lon from airports df
for city in list(df1.index):
    if city in list(airports_df['city']):
        df1.loc[city,'origin_one_latitude']=list(airports_df[airports_df['city']==city]['lat'])[0]
        df1.loc[city,'origin_one_longitude']=list(airports_df[airports_df['city']==city]['lon'])[0]

In [355]:
df1['origin_two_latitude'] = np.nan
df1['origin_two_longitude'] = np.nan

df1['origin_two_latitude'] = list(airports_df[airports_df['city']==city_one]['lat'])[0]
df1['origin_two_longitude'] = list(airports_df[airports_df['city']==city_one]['lon'])[0]

In [356]:
# creating a new columns from the index which will be used for the Destination column
df1 = df1.reset_index()

In [357]:
# rename index column to Destination
df1.rename(columns={'index':'Destination',f'total_stops_{city_one}':'Total_Stops'},inplace=True)

In [358]:
# create placeholder column for Route. This column will be dropped during preproc pipeline
df1['Route'] = df1["Source"] + "-" + df1["Destination"]

In [359]:
df1_formatted = df1[['Airline','Date_of_Journey','Source','Destination',
                     'Route','Dep_Time','Arrival_Time','Duration','Total_Stops',
                     'Additional_Info','origin_one_latitude','origin_one_longitude',
                    'origin_two_latitude','origin_two_longitude']]

In [360]:
df1_formatted.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,origin_one_latitude,origin_one_longitude,origin_two_latitude,origin_two_longitude
0,Multiple Carriers,01/04/2022,Sydney,Hanoi,Sydney-Hanoi,13:25,11:25,18h 0m,1,No info,21.221201,105.806999,-33.946098,151.177002
1,Multiple Carriers,01/04/2022,Sydney,Bangkok,Sydney-Bangkok,10:50,16:40,9h 50m,0,No info,13.681100,100.747002,-33.946098,151.177002
2,Multiple Carriers,01/04/2022,Sydney,Taipei,Sydney-Taipei,13:25,05:45,19h 0m,1,No info,25.077700,121.233002,-33.946098,151.177002
3,Multiple Carriers,01/04/2022,Sydney,Manila,Sydney-Manila,11:15,16:50,8h 20m,0,No info,14.508600,121.019997,-33.946098,151.177002
4,Multiple Carriers,01/04/2022,Sydney,Fukuoka,Sydney-Fukuoka,21:45,08:50,13h 5m,1,No info,33.585899,130.451004,-33.946098,151.177002


In [361]:
df1_formatted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Airline               16 non-null     object 
 1   Date_of_Journey       16 non-null     object 
 2   Source                16 non-null     object 
 3   Destination           16 non-null     object 
 4   Route                 16 non-null     object 
 5   Dep_Time              16 non-null     object 
 6   Arrival_Time          16 non-null     object 
 7   Duration              16 non-null     object 
 8   Total_Stops           16 non-null     object 
 9   Additional_Info       16 non-null     object 
 10  origin_one_latitude   16 non-null     float64
 11  origin_one_longitude  16 non-null     float64
 12  origin_two_latitude   16 non-null     float64
 13  origin_two_longitude  16 non-null     float64
dtypes: float64(4), object(10)
memory usage: 1.9+ KB


In [2]:
pipeline = joblib.load('../model.joblib')

AttributeError: Can't get attribute 'DateFormatter' on <module '__main__'>

In [86]:
df2 = matches_df.iloc[:,3:12:2].reset_index()

In [91]:
df1['Source'] = city_one

In [99]:
df1['Date_of_Departure'] = "/".join((df1[f'local_departure_{city_one}'][0][:10].split('-')[::-1]))

In [122]:
df1['Dep_time'] = np.nan

In [119]:
for _ in range(0,len(df1)):
    df1. df1[f"local_departure_{city_one}"]

0     2022-04-01T13:25:00.000Z
1     2022-04-01T10:50:00.000Z
2     2022-04-01T13:25:00.000Z
3     2022-04-01T11:15:00.000Z
4     2022-04-01T21:45:00.000Z
5     2022-04-01T08:15:00.000Z
6     2022-04-01T21:45:00.000Z
7     2022-04-01T06:25:00.000Z
8     2022-04-01T07:00:00.000Z
9     2022-04-01T16:00:00.000Z
10    2022-04-01T12:45:00.000Z
11    2022-04-01T19:05:00.000Z
12    2022-04-01T16:00:00.000Z
13    2022-04-01T16:00:00.000Z
14    2022-04-01T06:25:00.000Z
15    2022-04-01T16:00:00.000Z
Name: local_departure_Sydney, dtype: object

In [117]:
df1

,index,duration_Sydney,total_stops_Sydney,distance_Sydney,local_departure_Sydney,local_arrival_Sydney,Source,Date_of_Departure,Dep_Time
0,Hanoi,18.00,1,7798.15,2022-04-01T13:25:00.000Z,2022-04-02T11:25:00.000Z,Sydney,01/04/2022,13:25
1,Bangkok,9.83,0,7523.15,2022-04-01T10:50:00.000Z,2022-04-01T16:40:00.000Z,Sydney,01/04/2022,13:25
2,Taipei,19.00,1,7293.46,2022-04-01T13:25:00.000Z,2022-04-02T05:45:00.000Z,Sydney,01/04/2022,13:25
3,Manila,8.33,0,6269.74,2022-04-01T11:15:00.000Z,2022-04-01T16:50:00.000Z,Sydney,01/04/2022,13:25
4,Fukuoka,13.08,1,7823.25,2022-04-01T21:45:00.000Z,2022-04-02T08:50:00.000Z,Sydney,01/04/2022,13:25
5,Singapore,8.25,0,6301.43,2022-04-01T08:15:00.000Z,2022-04-01T16:50:00.000Z,Sydney,01/04/2022,13:25
6,Osaka,12.33,1,7790.50,2022-04-01T21:45:00.000Z,2022-04-02T08:05:00.000Z,Sydney,01/04/2022,13:25
7,Kuala Lumpur,8.83,0,6591.75,2022-04-01T06:25:00.000Z,2022-04-01T20:15:00.000Z,Sydney,01/04/2022,13:25
8,Ho Chi Minh City,8.92,0,6853.68,2022-04-01T07:00:00.000Z,2022-04-01T16:10:00.000Z,Sydney,01/04/2022,13:25
9,Da Nang,21.17,1,7190.96,2022-04-01T16:00:00.000Z,2022-04-02T15:20:00.000Z,Sydney,01/04/2022,13:25


In [116]:
df1["local_arrival_Sydney"][0][11:16]

'11:25'

In [88]:
df2['Source'] = city_two

,index,duration_Sydney,total_stops_Sydney,distance_Sydney,local_departure_Sydney,local_arrival_Sydney,Source
0,Hanoi,18.00,1,7798.15,2022-04-01T13:25:00.000Z,2022-04-02T11:25:00.000Z,Sydney
1,Bangkok,9.83,0,7523.15,2022-04-01T10:50:00.000Z,2022-04-01T16:40:00.000Z,Sydney
2,Taipei,19.00,1,7293.46,2022-04-01T13:25:00.000Z,2022-04-02T05:45:00.000Z,Sydney
3,Manila,8.33,0,6269.74,2022-04-01T11:15:00.000Z,2022-04-01T16:50:00.000Z,Sydney
4,Fukuoka,13.08,1,7823.25,2022-04-01T21:45:00.000Z,2022-04-02T08:50:00.000Z,Sydney
5,Singapore,8.25,0,6301.43,2022-04-01T08:15:00.000Z,2022-04-01T16:50:00.000Z,Sydney
6,Osaka,12.33,1,7790.50,2022-04-01T21:45:00.000Z,2022-04-02T08:05:00.000Z,Sydney
7,Kuala Lumpur,8.83,0,6591.75,2022-04-01T06:25:00.000Z,2022-04-01T20:15:00.000Z,Sydney
8,Ho Chi Minh City,8.92,0,6853.68,2022-04-01T07:00:00.000Z,2022-04-01T16:10:00.000Z,Sydney
9,Da Nang,21.17,1,7190.96,2022-04-01T16:00:00.000Z,2022-04-02T15:20:00.000Z,Sydney


In [44]:
date_filtered = matches_df['local_departure_Sydney'][0][:10].split('-')[::-1]

In [45]:
date_formatted = "/".join(date_filtered)